In [1]:
import damask
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
def orientations(file, how_many):
    
    ang_tab = damask.Table.load(file)
    phi1 = ang_tab.get('phi1')
    PHI = ang_tab.get('PHI')
    phi2 = ang_tab.get('phi2')
    weight = ang_tab.get('weight')

    phi = np.hstack((phi1, PHI, phi2))
    
    eur_list = []
    quat_list = []

    for cnt in range(how_many):
        quaternion = damask.Orientation.from_ODF(weights = weight, phi = phi, degrees=True, family = 'cubic')
        eur_list.append(damask.Rotation(quaternion).as_Euler_angles())
        quat_list.append(quaternion)
        
    # This function returns the two listings, eulr_list and quaterinian list 
    return eur_list, quat_list

In [3]:
file = 'takajo_55.ang'

In [4]:
ang_tab = damask.Table.load(file)

In [5]:
ang_tab


      phi1   PHI  phi2   weight
0      0.0   0.0   0.0  0.80139
1      5.0   0.0   0.0  0.89851
2     10.0   0.0   0.0  1.03010
3     15.0   0.0   0.0  0.95563
4     20.0   0.0   0.0  0.71735
...    ...   ...   ...      ...
6151  65.0  90.0  85.0  0.61791
6152  70.0  90.0  85.0  0.65936
6153  75.0  90.0  85.0  0.85311
6154  80.0  90.0  85.0  0.89951
6155  85.0  90.0  85.0  0.77346

[6156 rows x 4 columns]

In [6]:
test = orientations(file, 3)

In [7]:
len(test[1])

3

In [8]:
config_material = damask.ConfigMaterial()

In [9]:
config_material

homogenization: {}
phase: {}
material: []

In [10]:
# based on the upcoming contents in the config_material calss, 
# we need to defien three major things, homogenization scheme, 'homogenization'
# phase description 'phase'
# material informaiton 'material' 

#config_material['homogenization']['dummy'] = {'N_constituents':1, 'mechanical':{'type':'pass'}}
#config_material['phase']['BCC'] = damask.ConfigMaterial.load('mat00.yaml')

In [11]:
config_material = damask.ConfigMaterial.load('material_zero.yaml')

In [12]:
config_material

homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}
phase:
  BCC:
    lattice: cI
    mechanical:
      output: [F, P, F_e, F_p, L_p, O]
      elastic: {type: Hooke, C_11: 106750000000.0, C_12: 60410000000.0, C_44: 28340000000.0}
      plastic:
        type: phenopowerlaw
        N_sl: [12]
        a_sl: 2.25
        atol_xi: 1.0
        dot_gamma_0_sl: 0.001
        h_0_sl-sl: 75000000.0
        h_sl-sl: [1, 1, 1.4, 1.4, 1.4, 1.4, 1.4]
        n_sl: 20
        output: [xi_sl]
        xi_0_sl: [31000000.0]
        xi_inf_sl: [63000000.0]

In [13]:
config_material = config_material.material_add(homogenization='SX', phase = 'BCC', O = test[1])

In [14]:
#config_material['phase']['BCC']['mechanical']['plastic']['xi_0_sl'][0]*=1.2

In [15]:
len(config_material['material'])

3

In [16]:
config_material

homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}
phase:
  BCC:
    lattice: cI
    mechanical:
      output: [F, P, F_e, F_p, L_p, O]
      elastic: {type: Hooke, C_11: 106750000000.0, C_12: 60410000000.0, C_44: 28340000000.0}
      plastic:
        type: phenopowerlaw
        N_sl: [12]
        a_sl: 2.25
        atol_xi: 1.0
        dot_gamma_0_sl: 0.001
        h_0_sl-sl: 75000000.0
        h_sl-sl: [1, 1, 1.4, 1.4, 1.4, 1.4, 1.4]
        n_sl: 20
        output: [xi_sl]
        xi_0_sl: [31000000.0]
        xi_inf_sl: [63000000.0]
material:
- constituents:
  - phase: BCC
    O: [0.8848246737055843, 0.20338670806338613, -0.07402670778046234, 0.41260052147341175]
    v: 1.0
  homogenization: SX
- constituents:
  - phase: BCC
    O: [0.38268343236508984, 0.0, 0.0, 0.9238795325112867]
    v: 1.0
  homogenization: SX
- constituents:
  - phase: BCC
    O: [0.42169572290644275, 0.4869588190164687, 0.22707262651349336, 0.7303984174084459]
    v: 1.0
  homogenization

In [17]:
config_material.save('material.yaml')